In [1]:
import nltk
from nltk import bigrams
from nltk import trigrams
from nltk import FreqDist
from nltk.util import ngrams
from nltk.probability import LidstoneProbDist
from collections import Counter
from itertools import product
from math import log
from math import exp
from Bio.Seq import Seq
from Bio import SeqIO
import os;


ImportError: No module named 'sklearn.model_selection'

In [172]:
###### LOAD DATA ########
dataPath = '/Users/Akshay/Dropbox/code/GenomeDreaming/data/'

##Iterates through every FASTA file, parsing genome id (key) and sequence (value).
genomes = []
for fastaFile in os.listdir(dataPath):
    if "fna" in fastaFile: #ensures only fasta files being read
        handle = open(dataPath + fastaFile, "r")
        genomes.append(list(SeqIO.parse(handle, "fasta")))
        handle.close()
len(genomes[1][0].seq)



###### TRAIN N-gram Model ######
train = genomes[1][0].seq
test = "GGAAA"

n = 1; #define the n for this n-gram

#Laplacian smoothing for all combinations of n-gram; creates initial
#frequency table with freq of 1 given to every possible combination.
vocabulary = "ATCG"
smoothingVocab = ['A','T','C','G']
#perms = [''.join(p) for p in product(vocabulary,repeat = n)]
bgs_smooth = nltk.ngrams("",1)
fdist = nltk.FreqDist(bgs_smooth)
for p in product(vocabulary,repeat = n):
    bgs_smooth = nltk.ngrams(p,n)
    fdist += nltk.FreqDist(bgs_smooth)


#Calculate actual n-gram frequencies in training set and add to 
#laplacian smoothing
bgs_train = nltk.ngrams(train,n)
fdist += nltk.FreqDist(bgs_train)

#Convert n-gram frequencies to probabilities. |V| = 4^n is implicitly added to the denominator to account for laplacian smoothing,
#since fdist.values already have 1 added to each |V|.
totalVal = sum(fdist.values())
for key in fdist.keys():
    fdist[key] = fdist[key]/float(totalVal)
#fdist.values()

#Perplexity calculation. Remembe I need to add +V to bottom probability.
#Used log probabilities, so log perplexity calculation since risk of underflow with lots of probabilities
#Need to make sure I'm calculating the perplexity correctly for ngrams;
#specifically, the idea of using the previous n words to predict the next
#right now i'm just treating each consecutive n-gram in the test set
#as independent from the last and multiplying all their probabilities
#(sum here, because log)
def perplexity(testset, model):
    perplexity = 1
    #iterate through all possible n-grams; won't work for n=1
    for char in range(n, len(test)+1):
        ngram = tuple(test[char-n:char])
        perplexity = perplexity * (1/model.get(ngram))
    perplexity = pow(perplexity, 1/float()) 
    return perplexity

test = genomes[1][0].seq
perplexity = 0
##### CALCULATE TEST SEQUENCE PERPLEXITY #####
for char in range(n, len(test)+1):
    ngram = tuple(test[char-n:char])
    perplexity += log(1/fdist.get(ngram))
    #print(ngram)
    #print(fdist.get(ngram))
    #print(perplexity)
perplexity = exp(perplexity/(len(test)-n+1)) 
print(perplexity)

3.8908604251539196


In [264]:
Perplexity with 5-gram
805.0094611485741
1140.0463786516896

Perplexity w/ 2-gram
14.880629212981228
16.812607119054302

Perplexity w/ 1-gram
3.8908604251539196
4.133334511155625

['Monty', 'the', 'hamburger']

In [53]:
corpus ="""
Monty Python (sometimes known as The Pythons) were a British surreal comedy group who created the sketch comedy show Monty Python's Flying Circus,
that first aired on the BBC on October 5, 1969. Forty-five episodes were made over four series. The Python phenomenon developed from the television series
into something larger in scope and impact, spawning touring stage shows, films, numerous albums, several books, and a stage musical.
The group's influence on comedy has been compared to The Beatles' influence on music."""

In [57]:
import collections, nltk
# we first tokenize the text corpus
tokens = nltk.word_tokenize(corpus)

#here you construct the unigram language model 
def unigram(tokens):    
    model = collections.defaultdict(lambda: 0.01) #0.01 is laplace smoothing factor
    for f in tokens:
        try:
            model[f] += 1
        except KeyError:
            model [f] = 1
            continue
    for word in model:
        model[word] = model[word]/float(len(model))
    return model

def perplexity(testset, model):
    testset = testset.split()
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[word])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

testset1 = "Monty"
testset2 = "abracadabra gobbledygook rubbish"

model = unigram(tokens)
print(perplexity(testset1, model))
print(perplexity(testset2, model))

33.83084577114428
99.99999999999997


In [89]:
sentence = 'this is a foo bar sentences and i want to ngramize it'
n = 6
sixgrams = ngrams(sentence.split(), n)
for grams in sixgrams:
  print(grams)

('this', 'is', 'a', 'foo', 'bar', 'sentences')
('is', 'a', 'foo', 'bar', 'sentences', 'and')
('a', 'foo', 'bar', 'sentences', 'and', 'i')
('foo', 'bar', 'sentences', 'and', 'i', 'want')
('bar', 'sentences', 'and', 'i', 'want', 'to')
('sentences', 'and', 'i', 'want', 'to', 'ngramize')
('and', 'i', 'want', 'to', 'ngramize', 'it')


In [154]:
from nltk.model import MLENgramModel
from nltk.model import count_ngrams

text="abcdefghabc asdf asdf"
#Create your bigrams
bgs = nltk.ngrams(text,5)
#five_model = MLENgramModel(bgs)
vocab = build_vocabulary(1, text)
print(vocab)
bigram_counts = count_ngrams(2, vocab, text)
print(sorted(bigram_counts.ngrams[2].conditions()))
print(bigram_counts.ngrams[2][('f',)])
vocab
print(bigram_counts.unigrams)
#compute frequency distribution for all the bigrams in the text
bigram_model = MLENgramModel(bigram_counts)
ex_score = bigram_model.score("yawned", ["he"])
print(ex_score)

fdist = nltk.FreqDist(bgs)
for k,v in fdist.items():
   k

testset1 = "Monty"
testset2 = "abracadabra gobbledygook rubbish"




NgramModelVocabulary({'a': 4, 'd': 3, 'f': 3, 'b': 2, 'c': 2, ' ': 2, 's': 2, 'e': 1, 'g': 1, 'h': 1})
[(' ',), ('<s>',), ('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('s',)]
<FreqDist with 1 samples and 3 outcomes>
<FreqDist with 12 samples and 63 outcomes>
0


In [143]:
a = "a"

In [22]:
[a b c]

SyntaxError: invalid syntax (<ipython-input-22-b76c348e7bd1>, line 1)

In [48]:
gram = []
for Ngram in range (1,4):
    gram.append(1)
tuple(gram)

(1, 1, 1)

In [139]:
math.log(10)

2.302585092994046

In [112]:
pow(4,4)

256

In [141]:
exp(1)

NameError: name 'exp' is not defined

In [162]:
genomeDict = {};


1830138

In [2]:
from sklearn.model_selection import KFold


In [9]:
import pickle

favorite_color = { "lion": "yellow", "kitty": "red" }

pickle.dump( favorite_color, open( "save.p", "wb" ) )



In [10]:
fav = pickle.load( open( "save.p", "rb" ) )


In [11]:
fav

{'kitty': 'red', 'lion': 'yellow'}